In [2]:
import requests
from bs4 import BeautifulSoup as BS
url = "http://gs25.gsretail.com/gscvs/ko/store-services/locations"
payload = {"pageNum" :"1",
"pageSize" :"5",
"searchShopName" :"",
"searchSido" :"",
"searchGugun" :"",
"searchDong" :"",
"searchType" :"",
"searchTypeService" :"0",
"searchTypeToto" :"0",
"searchTypeCafe25" :"0",
"searchTypeInstant" :"0",
"searchTypeDrug" :"0",
"searchTypeSelf25" :"0",
"searchTypePost" :"0",
"searchTypeATM" :"0",
"searchTypeWithdrawal" :"0",
"searchTypeTaxrefund" :"0",
"searchTypeSmartAtm" :"0",
"searchTypeSelfCookingUtensils" :"0",
"searchTypeDeliveryService" :"0"}
r = requests.post(url,data = payload)
bs = BS(r.text)

In [3]:
bs

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="no-cache" http-equiv="pragma"/>
<meta content="No-Cache" http-equiv="Cache-Control"/>
<title>Gsretail Site</title>
<script src="/_ui/desktop/common/js/gsretail/jquery-1.11.2.js" type="text/javascript"></script>
<script src="/_ui/desktop/common/js/gsretail/ui.js" type="text/javascript"></script>
<script src="/_ui/desktop/common/js/gsretail/jPushMenu.js" type="text/javascript"></script>
<script src="/_ui/desktop/common/js/gsretail/selectbox-min.js" type="text/javascript"></script>
<link href="/_ui/desktop/common/css/gsretail/common.css" rel="stylesheet" type="text/css"/>
</head>
<body>
<div id="wrap">
<!-- Error Wrap -->
<div class="error_wrap">
<div class="

In [5]:
with requests.Session() as s:
    r = s.get(url)
    bs = BS(r.text)
    csrf = bs.find("form", id="CSRFForm").find("input")['value']
#세션 연결상태 유지 위해서 with구문 사용

In [6]:
csrf

'36e331f8-cdae-4452-b553-713cd3b18f6b'

In [7]:
import json
import pandas as pd
post_url = "http://gs25.gsretail.com/gscvs/ko/store-services/locationList?CSRFToken={}"
with requests.Session() as s:
    r = s.get(url)
    bs = BS(r.text)
    csrf = bs.find("form", id="CSRFForm").find("input")['value']
    r2 = s.post(post_url.format(csrf), data=payload)
    payload['pageSize'] = 5000

gs = pd.DataFrame(json.loads(r2.json())['results'])
gs

,offeringService,shopCode,longs,address,shopName,lat
0,"[toto, cafe25, post, withdrawal]",V3414,37.51826292,"경기 부천시 오정구 석천로345, 300동 114호(삼정동 365)",GS252테크노파크점,126.76388845
1,"[cafe25, drug, post, atm, withdrawal, smart_atm]",VX815,38.00414846,"강원 양양군 현북면 동해대로1242, 38선휴게소 (잔교리 41-10, 38휴게소)",GS2538선휴게소점,128.73283627
2,[post],VQ037,37.52831065,"서울 양천구 목동서로159-1 (목1동 917-1, CBS방송국1층)",GS25CBS점,126.87486792
3,[cafe25],V7W76,36.44932895,"대전 대덕구 대덕대로1447번길39, 1층 (문평동 140)",GS25CJ대전HUB점,127.40708036
4,NaN,V6Z47,37.69488283,"경기 고양시 일산동구 성현로615 (사리현동 151-1, CJ대한통운)",GS25CJ덕양SUB점,126.84958684


In [8]:
master = {x.text : x['value'] for x in bs.find("select", id="stb1").findAll("option")[1:]}

In [9]:
master

{'서울시': '11',
 '부산시': '26',
 '대구시': '27',
 '광주시': '29',
 '인천시': '28',
 '대전시': '30',
 '울산시': '31',
 '세종특별자치시': '36',
 '경기도': '41',
 '강원도': '42',
 '충청남도': '44',
 '충청북도': '43',
 '경상남도': '48',
 '경상북도': '47',
 '전라남도': '46',
 '전라북도': '45',
 '제주특별자치도': '50'}

In [10]:
import json
import pandas as pd
post_url = "http://gs25.gsretail.com/gscvs/ko/store-services/locationList?CSRFToken={}"
total = []
with requests.Session() as s:
    r = s.get(url)
    bs = BS(r.text)
    csrf = bs.find("form", id="CSRFForm").find("input")['value']
    for code in master.values():
        payload['pageSize'] = 5000
        payload['searchSido'] = code
        r2 = s.post(post_url.format(csrf), data=payload)
        total.append(pd.DataFrame(json.loads(r2.json())['results']))
gs = pd.concat(total)
gs

,offeringService,shopCode,longs,address,shopName,lat
0,[post],VQ037,37.52831065,"서울 양천구 목동서로159-1 (목1동 917-1, CBS방송국1층)",GS25CBS점,126.87486792
1,"[cafe25, post]",VO395,37.5783899,"서울 마포구 매봉산로75, 1층 107호 (상암동 1610)",GS25DDMC점,126.89294205
2,"[cafe25, post]",VER76,37.57410234,"서울 서대문구 가재울미래로2, 203동 101호 (남가좌동 385, 2단지)",GS25DMC가재울점,126.91639401
3,"[cafe25, instant, post]",V9A48,37.50235508,"서울 강남구 테헤란로223, 1층 101호 (역삼동 677-25, 큰길타워빌딩)",GS25DXLAB점,127.04069826
4,"[drug, withdrawal]",VI989,37.55513349,서울 강동구 고덕로39 (암사3동 441-14),GS25G고덕점,127.12956973
...,...,...,...,...,...,...
384,"[cafe25, instant, drug, post]",V9377,33.51836478,제주 제주시 동화로1길49-9 (화북일동 13-6),GS25화북주공점,126.57739082
385,"[cafe25, post]",V4S83,33.52321296,제주 제주시 진남로33 (화북일동 4201-2),GS25화북진남점,126.56390997
386,"[cafe25, instant, withdrawal]",VY067,33.24402459,제주 서귀포시 안덕면 화순중앙로94 (화순리 358-3),GS25화순아일랜드점,126.33601411
387,"[cafe25, drug, withdrawal, self_cook]",VDB29,33.24653847,"제주 서귀포시 안덕면 화순로142, C동 1층 (화순리 1055-3)",GS25화순육거리점,126.33119176


In [11]:
gs['시'] = gs['address'].apply(lambda x : x.split()[0])

In [12]:
gs['시'].value_counts()

경기       4487
서울       3105
경남       1021
부산        909
인천        904
경북        807
충남        764
강원        660
대구        639
충북        531
전북        469
대전        459
전남        443
제주        387
광주        354
울산        332
세종        102
부산광역시       2
제주특별자       2
Name: 시, dtype: int64

## offering service 데이터 다루기

In [13]:
gs['offeringService']

0                                     [post]
1                             [cafe25, post]
2                             [cafe25, post]
3                    [cafe25, instant, post]
4                         [drug, withdrawal]
                       ...                  
384            [cafe25, instant, drug, post]
385                           [cafe25, post]
386            [cafe25, instant, withdrawal]
387    [cafe25, drug, withdrawal, self_cook]
388      [cafe25, instant, drug, withdrawal]
Name: offeringService, Length: 16377, dtype: object

In [14]:
gs['offeringService'].isnull().sum()

423

In [16]:
gs.loc[gs['offeringService'].notnull(), '의료'] = gs[gs['offeringService'].notnull()]['offeringService'].apply(lambda x :
                                    'Y' if 'drug' in x else 'N') #의료 관련 서비스가 있는지 없는지 표시

In [17]:
gs

,offeringService,shopCode,longs,address,shopName,lat,시,의료
0,[post],VQ037,37.52831065,"서울 양천구 목동서로159-1 (목1동 917-1, CBS방송국1층)",GS25CBS점,126.87486792,서울,N
1,"[cafe25, post]",VO395,37.5783899,"서울 마포구 매봉산로75, 1층 107호 (상암동 1610)",GS25DDMC점,126.89294205,서울,N
2,"[cafe25, post]",VER76,37.57410234,"서울 서대문구 가재울미래로2, 203동 101호 (남가좌동 385, 2단지)",GS25DMC가재울점,126.91639401,서울,N
3,"[cafe25, instant, post]",V9A48,37.50235508,"서울 강남구 테헤란로223, 1층 101호 (역삼동 677-25, 큰길타워빌딩)",GS25DXLAB점,127.04069826,서울,N
4,"[drug, withdrawal]",VI989,37.55513349,서울 강동구 고덕로39 (암사3동 441-14),GS25G고덕점,127.12956973,서울,Y
...,...,...,...,...,...,...,...,...
384,"[cafe25, instant, drug, post]",V9377,33.51836478,제주 제주시 동화로1길49-9 (화북일동 13-6),GS25화북주공점,126.57739082,제주,Y
385,"[cafe25, post]",V4S83,33.52321296,제주 제주시 진남로33 (화북일동 4201-2),GS25화북진남점,126.56390997,제주,N
386,"[cafe25, instant, withdrawal]",VY067,33.24402459,제주 서귀포시 안덕면 화순중앙로94 (화순리 358-3),GS25화순아일랜드점,126.33601411,제주,N
387,"[cafe25, drug, withdrawal, self_cook]",VDB29,33.24653847,"제주 서귀포시 안덕면 화순로142, C동 1층 (화순리 1055-3)",GS25화순육거리점,126.33119176,제주,Y


In [18]:
gs['의료'].value_counts(normalize=True)

Y    0.826689
N    0.173311
Name: 의료, dtype: float64

In [19]:
gs[gs['의료']=='Y'].groupby(['시'])['의료'].count().sum()

13189

In [20]:
gs['의료'] = gs['의료'].fillna('N')

## 모든 서비스 제공하는 편의점 있는지 확인

In [21]:
gs[gs['offeringService'].notnull()][gs[gs['offeringService'].notnull()]['offeringService'].apply(lambda x :len(x) == 11)]

,offeringService,shopCode,longs,address,shopName,lat,시,의료
165,"[toto, cafe25, instant, drug, post, atm, withd...",V9373,33.489818,제주 제주시 도령로76(연동 251-57),GS25연동신광점,126.48660204,제주,Y
